In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import algos as al
import func

rng = np.random.default_rng(424)

In [ ]:
def plot_targets(targets):
    rows = len(targets)
    cols = len(targets[0])

    fig, axes = plt.subplots(
        nrows=rows, ncols=cols, figsize=(16, 9), subplot_kw=dict(projection="3d")
    )
    fig.tight_layout()

    for r in range(rows):
        for c in range(cols):
            for axis in [axes[r][c].xaxis, axes[r][c].yaxis, axes[r][c].zaxis]:
                axis.set_ticklabels([])
            targets[r][c].plot(axes[r][c], 256, cmap=mpl.cm.Oranges)

    fig.savefig("fig/targets.png", dpi=300, transparent=True)


ackley = func.ackley(2)
griewank = func.griewank(2)
schwefel = func.schwefel(2)
skin = func.skin(2)
forest = func.forest(2)
megacity = func.megacity(2)

plot_targets([[ackley, griewank, schwefel], [skin, forest, megacity]])

In [ ]:
def benchmark(func, algos, n_iters=50, eval_calls_lim=1000):
    fig, axes = plt.subplots()
    lines = []
    legend = []

    for algo in algos:
        anchors = []

        for i in range(n_iters):
            anchors_dict = algo.optimize(func, eval_calls_lim)
            for item in anchors_dict.items():
                anchors.append(item)

        anchors.sort(key=lambda x: x[0])
        x = []
        y = []
        i = 0
        while i < len(anchors):
            x.append(anchors[i][0])
            value_sum = 0
            j = i + 1
            while j < len(anchors) and anchors[j][0] == x[-1]:
                value_sum += anchors[j][1]
                j += 1

            y.append(value_sum / (j - i))
            i = j

        line = axes.plot(x, y)
        lines.append(line[0])
        legend.append(algo.__class__.__name__)

    axes.legend(tuple(lines), tuple(legend), loc="upper right")

    axes.set_xlabel("Function evaluations")
    axes.set_ylabel("Found minimum")
    axes.grid()
    
    title = f"Comparison on {func.name} function"
    axes.set_title(title)
    fig.savefig(f"benchmarks/{title}.png", dpi=300)

In [ ]:
algos = (
    al.RandomSearch(),
    al.ParticleSwarm(
        n_particles=20, inertia_cf=0.721, cognitive_cf=1.193, social_cf=1.193
    ),
    al.BeeHive(
        shrink_cf=0.9,
        n_scouts=34,
        n_elites=7,
        n_extras=2,
        n_elite_sites=1,
        n_extra_sites=2,
    ),
    al.AntHill(n_ants=30, pher_cf=1, dist_cf=1, skip_cf=1.2),
)
benchmark(
    schwefel,
    algos=algos
)